In [ ]:
import pandas as pd
import json
import ast


f = open('./poec_lang.us.json')
translations = json.load(f)

f = open('./poec_data.json')
data = json.load(f)

In [13]:
mod_names = translations["mod"] # all possible mod en names
base_names = translations["base"] # all possible mod en names

mod_group_names = translations["mgroup"] # all possible mod en names

print(data.keys())
print(translations.keys())

bases =  data["bases"]['seq'] # all possible bases
bases_mods =  data["basemods"] # list of mods for all possible bases
tiers = data["tiers"]

dict_keys(['bitems', 'bases', 'bgroups', 'modifiers', 'mgroups', 'mtypes', 'fossils', 'catalysts', 'essences', 'maeven', 'aliases', 'tiers', 'basemods', 'modbases', 'mdefs', 'clngs'])
dict_keys(['base', 'bgroup', 'mod', 'mdef', 'mgroup', 'mtype', 'fossil', 'catalyst', 'essence', 'bitem', 'maven'])


In [ ]:
item_mods = {}

for base in bases:
    print(base)
    if base["name_base"].find("Watchstone") != -1 or base["name_base"] == "Gear" or base["name_base"].find("Cluster Jewel") != -1 or base["name_base"] == "Tools":
        continue
    base_group_id = base["id_bgroup"]
    base_id = base["id_base"]
    ########################## BASE NAMES
    base_name = translations["base"][base_id]
    base_group_name = translations["bgroup"][base_group_id]

    # print(base_group_name, base_name)
    ########################## BASE MODS
    local_item_mods = {}

    curr_base_mods_ids = bases_mods[base_id] # ids of current base's available mods
    base_mods_grouped_tiers = {} # all range values for all tiers of all mods

    for curr_mod_id in curr_base_mods_ids:
        mod_name = mod_names[curr_mod_id]

        if not mod_name in base_mods_grouped_tiers:
            base_mods_grouped_tiers[mod_name] = []

        if not mod_name in local_item_mods:
            local_item_mods[mod_name] = {"tiers": []}

        mod_tiers = tiers[curr_mod_id] # all tiers for current mod
        mod_tiers_for_curr_base = mod_tiers[base_id] # all available current mod tiers for current base 

        for tier in mod_tiers_for_curr_base: # iterate over each available tier of mod for current base
            local_item_mods[mod_name]["tiers"].append({"values": tier["nvalues"], "weight": tier["weighting"]})
            base_mods_grouped_tiers[mod_name].append(tier["nvalues"])

    for mod_name in base_mods_grouped_tiers.keys():
        result = []
        for s in base_mods_grouped_tiers[mod_name]:
            lst_of_lst = ast.literal_eval(s)
            for lst_of_num in lst_of_lst:
                if type(lst_of_num) != list:
                    result.append(lst_of_num)
                    continue
                for num in lst_of_num:
                    result.append(num)
        if len(result) > 0:
            local_item_mods[mod_name]["min"] = min(result)
            local_item_mods[mod_name]["max"] = max(result)
            #print(mod_name, result, min(result), max(result), local_item_mods[mod_name]["min"])
            print()
        else:
            local_item_mods[mod_name]["min"] = 0
            local_item_mods[mod_name]["max"] = 1




        #print(curr_mod_id, mod_name, "\n", mod_tiers_for_curr_base)
        #print(mod_tiers_for_curr_base)
    item_mods[base_name] = local_item_mods
    # print("local wtf",local_item_mods, "FDSGHJKFSDHJKF", item_mods[base_name])
    print()



with open("res.json", 'w') as json_file:
    print(json.dumps(item_mods), file=json_file)

In [ ]:
item_mods_2 = {}

all_bases = data["bitems"]["seq"]

for base in all_bases:
    if base["name_bitem"].find("Watchstone") != -1 or base["name_bitem"] == "Gear" or base["name_bitem"].find("Cluster Jewel") != -1 or base["name_bitem"] == "Tools":
        continue
    base_id = base["id_base"]
    name = base["name_bitem"]

    el = [x for x in bases if x["id_base"] == base_id]
    item_mods_2[name] = {"base": el[0]["name_base"], "mods": item_mods[el[0]["name_base"]]}


# for base in bases:
#     print(base)
#     if base["name_base"].find("Watchstone") != -1 or base["name_base"] == "Gear" or base["name_base"].find("Cluster Jewel") != -1 or base["name_base"] == "Tools":
#         continue

#     base_group_id = base["id_bgroup"]
#     base_id = base["id_base"]
#     ########################## BASE NAMES
#     base_name = translations["base"][base_id]
#     base_group_name = translations["bgroup"][base_group_id]

#     base_full_name = data["basemods"]["2"]
#     #print(base_full_name, data["aliases"])


#     break
with open("res_full.json", 'w') as json_file:
    print(json.dumps(item_mods_2), file=json_file)